### Sharing Data Between Processes

The example below shows that processes use different address space. The <i>result</i> variable is not changed in the main process, but only within the <i>calc_square</i> function in the subprocess.

In [26]:
from multiprocessing import Process, Array, Value, Queue, Manager


In [13]:
result = []

def calc_square(numbers):
    global result
    for n in numbers:
        result.append(n*n)
    print(f'inside process: {str(result)}')
    
def demo_without_shared_memory():
    numbers = [2, 3, 4, 5, 8, 19]
    p = Process(target=calc_square, args=(numbers,))
    p.start()
    p.join()
    print(f'outside process: {str(result)}')
    
demo_without_shared_memory()


inside process: [4, 9, 16, 25, 64, 361]
outside process: []


#### Shared Data using Array

In [20]:
def calc_square_using_array(numbers, result):
    for ind, n in enumerate(numbers):
        result[ind] = n*n
    print(f'inside process: {result[:]}')

def demo_shared_data_using_array():
    
    numbers = [2, 3, 4, 5, 8, 19]
    
    # create an array of integers with size len(numbers)
    # and pass it to the process args
    result = Array('i', len(numbers)) 
    p = Process(target=calc_square_using_array, args=(numbers, result,))
    p.start()
    p.join()
    print(f'outside process: {result[:]}')

demo_shared_data_using_array()

inside process: [4, 9, 16, 25, 64, 361]
outside process: [4, 9, 16, 25, 64, 361]


#### Shared Data using Value

In [21]:

def foo(val):
    val.value = 11.24
    print(f'value inside process: {val.value}')
    
def demo_shared_data_using_value():
    val = Value('d', 3.14)
    p = Process(target=foo, args=(val,))
    p.start()
    p.join()
    print(f'value outside process: {val.value}')
    
demo_shared_data_using_value()

value inside process: 11.24
value outside process: 11.24


#### Shared Data using Queue

Note that the queue from multiprocessing.Queue is different from the one in queue.Queue. The multiprocessing queue lives in **shared memory** and is used to share data between processes. The Queue from the queue module lives in in-process memory and is used to share data between threads.

In [25]:
def calc_square_using_queue(numbers, queue):
    for number in numbers:
        queue.put(number * number)

def demo_shared_data_using_queue():
    
    numbers = [2, 3, 4, 5, 8, 19]
    
    queue = Queue()
    p = Process(target=calc_square_using_queue, args=(numbers, queue,))
    p.start()
    p.join()
    
    while not queue.empty():
        print(queue.get(), end=' ')

demo_shared_data_using_queue()

4 9 16 25 64 361 

### Shared Data using Manager

In [28]:
def bar(dictionary):
    dictionary['a'] = 10
    dictionary['b'] = 20
    dictionary['c'] = 30
    print(f'dictionary within process: {dictionary}')
    
def demo_shared_data_using_manager():
    
    manager = Manager()
    dictionary = manager.dict()
    dictionary['a'] = 0
    p = Process(target=bar, args=(dictionary,))
    p.start()
    p.join()
    print(f'dictionary outside process: {dictionary}')
    
demo_shared_data_using_manager()

dictionary within process: {'a': 10, 'b': 20, 'c': 30}
dictionary outside process: {'a': 10, 'b': 20, 'c': 30}
